In [1]:
import os
import warnings
import argparse
import datetime
import pathlib
import sys

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
import pickle

from test import test
from utils.evaluate_uncertainty import evaluate_uncertainty
from core.config import create_config, save_config
# from core.dataset import COCODataset
from core.dataset_hotswapped import COCODataset
from core.model import Model
from core.metrics import AccuracyLogger
import pickle
from SupContrast.losses import SupConLoss
import torch.nn.functional as F
## Initialization

In [2]:
import matplotlib.pyplot as plt

In [3]:
with open('args_hotswapped.p','rb') as File:
    args = pickle.load(File)
args.batch_size = 15
print('Hard mining set to:%s'%(args.hard_mining))

Hard mining set to:none


In [4]:
cfg = create_config(args)

arbitrary_material_fols = sorted([i for i in os.listdir('../openrooms/') if 'arbitrary' in i])
# new_materials = arbitrary_material_fols[:args.num_materials]
material_fols = [i for i in arbitrary_material_fols if 'repeat' in i]
#  = []
print('Using materials: %s'%material_fols)

# dataset = COCODataset(cfg.annotations, cfg.imagedir, (224,224), material_fols, normalize_means=[0.485, 0.456, 0.406], normalize_stds=[0.229, 0.224, 0.225])
dataset = COCODataset(cfg.annotations, cfg.imagedir, (224,224), material_fols)
dataloader = DataLoader(dataset, batch_size=4, num_workers=4, shuffle=True, pin_memory=True, drop_last=True)

Using materials: ['main_xml1_arbitrary_many_styles_repeat_0', 'main_xml1_arbitrary_many_styles_repeat_1', 'main_xml1_arbitrary_many_styles_repeat_2', 'main_xml1_arbitrary_many_styles_repeat_3', 'main_xml1_arbitrary_many_styles_repeat_4', 'main_xml1_arbitrary_many_styles_repeat_5', 'main_xml1_arbitrary_many_styles_repeat_6', 'main_xml1_arbitrary_many_styles_repeat_7']
Hotswapped material loader being used.
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44}
-------------------------------
Annotation Counts
-------------------------------
chair                     33800
cabinet                    9557
bathtub                    5745
desk                       8890
bookshelf                  6076
file_ca

/n/home05/smadan/.conda/envs/domain_adaptation/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
# for d in dataset[:5]:
#     print(d)

In [5]:
for data in dataloader[:5]:
    break

TypeError: 'DataLoader' object is not subscriptable

In [5]:
NUM_CLASSES = dataset.NUM_CLASSES
cfg.num_classes = NUM_CLASSES
save_config(cfg, args.outdir)
print(cfg)

model = Model.from_config(cfg)

#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)

#assert(model.TARGET_IMAGE_SIZE == model.CONTEXT_IMAGE_SIZE == dataset.image_size), "Image size from the dataset is not compatible with the encoder."

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate)



annotations: ../openrooms/annotation_files/main_xml1_paired_segregated_train.json
batch_size: 15
checkpoint: output_openrooms/style_training_8/checkpoint_7.tar
git: 7b079d31dd8717a1abda8002b5f9656d82a5055c
imagedir: ../openrooms/
imbalance_reweighting: false
learning_rate: 1.0e-05
num_classes: 45
num_decoder_heads: 8
num_decoder_layers: 6
test_annotations: null
test_imagedir: null
uncertainty_gate_type: learned
uncertainty_threshold: 0.8
weighted_prediction: false



In [6]:
if cfg.imbalance_reweighting:
    class_weights = torch.true_divide(dataset.relative_annotation_counts.max(), dataset.relative_annotation_counts)
    criterion = nn.CrossEntropyLoss(weight= class_weights.to(device))
else:
    criterion = nn.CrossEntropyLoss()

criterion_2 = SupConLoss(temperature=0.1)
criterion_3 = torch.nn.MSELoss()

In [7]:
for i, (context_images, target_images, context_images_2, target_images_2, bbox, labels_cpu) in enumerate(dataloader):
    labels = labels_cpu
    context_images_all = torch.cat([context_images, context_images_2], dim=0)
    target_images_all = torch.cat([target_images, target_images_2], dim=0)
    bbox_all = torch.cat([bbox, bbox], dim=0)
    labels_all = torch.cat([labels, labels], dim=0)
    labels_all_cpu = torch.cat([labels_cpu, labels_cpu], dim=0)

    output_uncertainty_branch , output_main_branch, output_weighted, uncertainty = model(context_images_all, target_images_all, bbox_all)
    break

In [8]:
output_main_branch

torch.Size([8, 45])